In [1]:
import ast 
import pandas as pd

df_usage = pd.read_csv('data/baseline.txt', sep='\t')
df_usage['app_seq'] = df_usage['app_seq'].apply(ast.literal_eval)
df_usage['recent_apps'] = df_usage['recent_apps'].apply(ast.literal_eval)
df_usage['time'] = df_usage['time'].apply(lambda x: str(x))

### 切分训练测试集

In [2]:
from sklearn.model_selection import train_test_split

df_mru = df_usage.copy()
df_mru_train, df_mru_test = train_test_split(df_mru, test_size=0.2, random_state=2021, stratify=df_mru['user'])
df_mru_test.head()

,user,time,app,app_seq,recent_apps
324648,722,201604250612,237,"[80, 1554, 411, 1889]","[54, 46, 1353, 1466, 1, 392, 80, 1554, 411, 1889]"
275431,667,201604211033,1,"[1, 5, 371, 1]","[1, 1, 1, 72, 1, 1, 1, 5, 371, 1]"
659336,942,201604210952,384,"[208, 29, 381, 389]","[252, 105, 210, 126, 202, 287, 208, 29, 381, 389]"
902536,994,201604261718,5,"[379, 70, 688, 586]","[1, 587, 126, 948, 255, 688, 379, 70, 688, 586]"
245546,615,201604231510,2,"[77, 77, 2, 202]","[426, 1, 2, 77, 202, 2, 77, 77, 2, 202]"


In [3]:
df_mru_test.shape

(181754, 5)

### 用MRU对测试集预测，计算Recall@1，Recall@5，Recall@10

In [4]:
import tqdm

correct = [0, 0, 0]

for i in tqdm.tqdm(range(len(df_mru_test))):
    app = df_mru_test.iloc[i]['app']
    mru = df_mru_test.iloc[i]['recent_apps']

    if app == mru[-1]:
        correct = [x+1 for x in correct]
    elif app in mru[-5:]:
        correct[1] += 1
        correct[2] += 1
    elif app in mru:
        correct[2] += 1
    else:
        pass

total = len(df_mru_test)
print("Recall@1: {}".format(correct[0] / total))
print("Recall@5: {}".format(correct[1] / total))
print("Recall@10: {}".format(correct[2] / total))

100%|██████████| 181754/181754 [00:38<00:00, 4778.65it/s]

Recall@1: 0.13939720721414658
Recall@5: 0.37022018772626736
Recall@10: 0.4740088251152657
